In [4]:
!pip install pyspark

You should consider upgrading via the '/Users/seyda.ozdemir/.conda/envs/Downloads/bin/python -m pip install --upgrade pip' command.


# Examples Of Pyspark ML

In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml').getOrCreate()

## Read the Dataset

In [3]:
tips = spark.read.csv('tips.csv', header=True, inferSchema = True)
tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [4]:
tips.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [5]:
tips.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

## Linear Regression

### Handling categorical features

In [6]:
# Handling categorical features
from pyspark.ml.feature import StringIndexer

In [7]:
indexer = StringIndexer(inputCol = "sex", outputCol='sex_indexed')
df = indexer.fit(tips).transform(tips)
df.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [8]:
#Categorical features convert to indexed
indexer = StringIndexer(inputCols = ["smoker","day","time"], outputCols =["smoker_indexed","day_indexed","time_indexed"])
df = indexer.fit(df).transform(df)
df.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [9]:
df.columns

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [10]:
#Create independent features using vector assembler
from pyspark.ml.feature import VectorAssembler

featureassembler = VectorAssembler(inputCols=['tip','size','sex_indexed', 'smoker_indexed','day_indexed', 'time_indexed'],
                                   outputCol= "Independent Features")
output = featureassembler.transform(df)

In [11]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|S

In [12]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [13]:
finalized_data = output.select("Independent Features", "total_bill")

In [14]:
finalized_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [15]:
#import Linearregression package
from pyspark.ml.regression import LinearRegression

In [16]:
#train test split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])


In [17]:
#Create regression model
regressor = LinearRegression(featuresCol = 'Independent Features', labelCol='total_bill')
regressor = regressor.fit(train_data)

In [18]:
regressor.coefficients

DenseVector([3.1626, 3.5021, -1.3394, 3.0433, 0.1184, -1.0225])

In [19]:
regressor.intercept

0.8854550897054642

In [20]:
# Create Prediction
pred_result = regressor.evaluate(test_data)

In [21]:
pred_result.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
| (6,[0,1],[2.0,2.0])|     12.69|14.214932468477736|
|(6,[0,1],[2.72,2.0])|     13.28| 16.49202677769766|
| (6,[0,1],[3.0,2.0])|      14.0|17.377563453505406|
| (6,[0,1],[3.0,4.0])|     20.45|24.381778862222333|
| (6,[0,1],[3.6,3.0])|     24.06|22.777249748880475|
| (6,[0,1],[4.3,2.0])|      21.7| 21.48898373404138|
| (6,[0,1],[5.0,3.0])|     31.27|27.204933127919215|
|(6,[0,1],[6.73,4.0])|     48.27| 36.17839243637555|
|(6,[0,1],[7.58,4.0])|     39.42|38.866628773649076|
| (6,[0,1],[9.0,4.0])|     48.33|43.357564772388365|
|[1.0,1.0,1.0,0.0,...|      7.25|6.2107586583535275|
|[1.0,2.0,0.0,1.0,...|      12.6|14.095590313618093|
|[1.1,2.0,1.0,1.0,...|      12.9|13.072418291382789|
|[1.36,3.0,1.0,0.0...|     18.64|13.567908621608817|
|[1.44,2.0,0.0,0.0...|      7.56| 11.65824651679064|
|[1.48,2.0,0.0,0.0...|      8.52|11.7847517561

/Users/seyda.ozdemir/.conda/envs/Downloads/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [22]:
print('Linear Regression Model R Squared(R2) on test data: ', pred_result.r2)

Linear Regression Model R Squared(R2) on test data:  0.6288466085749935


In [23]:
print('Linear Regression Model Mean Absolute Error(MAE) on test data: ', pred_result.meanAbsoluteError)

Linear Regression Model Mean Absolute Error(MAE) on test data:  4.5894155740110545


In [24]:
print('Linear Regression Model Mean Squared Error(MSE) on test data: ', pred_result.meanSquaredError)

Linear Regression Model Mean Squared Error(MSE) on test data:  34.04468655177479
